In [1]:
import numpy as np
import h5py
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from scipy import stats
import logging
import os

def load_and_preprocess_data(hdf5_file, file_idx, spatial_idx, time_start_idx, time_end_idx):
    """
    Load and preprocess data from the HDF5 file.
    
    Parameters:
        hdf5_file (str): Path to the HDF5 file.
        file_idx (int): Index of the file.
        spatial_idx (int): Index of the spatial chunk to load.
        time_start_idx (int): Start index for time chunks.
        time_end_idx (int): End index for time chunks (inclusive).
    Returns:
        raw_data (np.array): Raw data (5000x100).
        fft_data (np.array): FFT data (5000x100) (magnitude spectrum, normalized).
    """
    with h5py.File(hdf5_file, 'r') as f:
        raw_data = []
        for t in range(time_start_idx, time_end_idx + 1):
            chunk_name = f'chunk_{file_idx}_{t}_{spatial_idx}'
            chunk = f[chunk_name][:]
            raw_data.append(chunk)
    
    raw_data = np.concatenate(raw_data, axis=0)  # Combine time chunks
    
    # Perform FFT on the concatenated time chunk
    fft_data = np.abs(np.fft.fft(raw_data, axis=0))  # FFT along time axis
    
    # Normalize raw and FFT data
    raw_mean, raw_std = np.mean(raw_data), np.std(raw_data)
    raw_data = (raw_data - raw_mean) / (raw_std + 1e-6)
    
    fft_mean, fft_std = np.mean(fft_data), np.std(fft_data)
    fft_data = (fft_data - fft_mean) / (fft_std + 1e-6)
    
    return raw_data, fft_data

def detect_anomalies(hdf5_file, model_path, output_dir='anomaly_results'):
    """
    Detect anomalies in DAS data using autoencoder reconstruction error.
    
    Parameters:
        hdf5_file (str): Path to the HDF5 file.
        model_path (str): Path to the trained autoencoder model.
        output_dir (str): Directory to save anomaly results.
    """

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(message)s')
    
    # Load trained autoencoder
    autoencoder = keras.models.load_model(model_path)
    
    # Create output directory
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Prepare to track anomalies
    anomalies = []
    
    # Iterate through data sections
    with h5py.File(hdf5_file, 'r') as raw_f:
        num_files = len(raw_f.keys()) // (87 * 30) - 9
        for file_idx in range(num_files):
            for spatial_idx in range(87):
                print(spatial_idx)
                for time_idx in range(0, 27, 3):
                    # Load and preprocess data
                    raw_data, fft_data = load_and_preprocess_data(
                        hdf5_file, file_idx, spatial_idx, time_idx, time_idx + 4
                    )
                    
                    # Prepare data for reconstruction
                    raw_input = raw_data[np.newaxis, ...]
                    fft_input = fft_data[np.newaxis, ...]
                    
                    # Reconstruct signals
                    raw_recon = autoencoder.predict(raw_input, verbose=0)[0]
                    fft_recon = autoencoder.predict(fft_input, verbose=0)[0]
                    
                    # Calculate MSE for time and frequency domains
                    raw_mse = np.mean((raw_data - raw_recon)**2)
                    fft_mse = np.mean((fft_data - fft_recon)**2)
                    
                    anomalies.append({
                        'file_idx': file_idx,
                        'spatial_idx': spatial_idx,
                        'time_idx': time_idx,
                        'raw_mse': raw_mse,
                        'fft_mse': fft_mse,
                        'raw_data': raw_data,
                        'fft_data': fft_data,
                        'raw_recon': raw_recon,
                        'fft_recon': fft_recon
                    })

    # Log the final message
    logging.info("Anomaly detection complete. Results saved in %s", output_dir)
    
    # Calculate anomaly scores using standard deviation
    raw_mses = [a['raw_mse'] for a in anomalies]
    fft_mses = [a['fft_mse'] for a in anomalies]
    
    raw_z_scores = np.abs(stats.zscore(raw_mses))
    fft_z_scores = np.abs(stats.zscore(fft_mses))
    
    # Combine Z-scores (you can adjust weighting if needed)
    combined_z_scores = (raw_z_scores + fft_z_scores) / 2
    
    for a, z_score in zip(anomalies, combined_z_scores):
        a['z_score'] = z_score
    
    # Sort anomalies by z-score
    sorted_anomalies = sorted(anomalies, key=lambda x: x['z_score'], reverse=True)
    
    '''# Visualize top 5 and bottom 5 anomalies
    def plot_anomaly(anomaly, idx, is_top=True):
        plt.figure(figsize=(15, 10))
        
        # Time domain plot
        plt.subplot(2, 1, 1)
        plt.title(f'{"Top" if is_top else "Bottom"} Anomaly {idx+1} - Time Domain\n'
                  f'File: {anomaly["file_idx"]}, Spatial: {anomaly["spatial_idx"]}, '
                  f'Z-Score: {anomaly["z_score"]:.2f}')
        plt.plot(anomaly['raw_data'], label='Original')
        plt.plot(anomaly['raw_recon'], label='Reconstructed', linestyle='--')
        plt.legend()
        
        # Frequency domain plot
        plt.subplot(2, 1, 2)
        plt.title(f'{"Top" if is_top else "Bottom"} Anomaly {idx+1} - Frequency Domain')
        plt.plot(anomaly['fft_data'], label='Original FFT')
        plt.plot(anomaly['fft_recon'], label='Reconstructed FFT', linestyle='--')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 
                    f'{"top" if is_top else "bottom"}_anomaly_{idx+1}.png'))
        plt.close()
    
    # Plot top 5 and bottom 5 anomalies
    for i in range(5):
        plot_anomaly(sorted_anomalies[i], i, is_top=True)
        plot_anomaly(sorted_anomalies[-(i+1)], i, is_top=False)
    
    # Save anomaly details to CSV
    import pandas as pd
    anomaly_df = pd.DataFrame(sorted_anomalies)
    anomaly_df.to_csv(os.path.join(output_dir, 'anomaly_details.csv'), index=False)
    
    print("Anomaly detection complete. Results saved in", output_dir)'''


    import os
    import matplotlib.pyplot as plt
    import pandas as pd
   # import numpy as np
    
    # Define function to plot an anomaly
    def plot_anomaly(anomaly, idx):
        plt.figure(figsize=(15, 10))
    
        # Time domain plot 
        plt.subplot(2, 1, 1)
        plt.title(f'Anomaly {idx+1} - Time Domain\n'
                  f'File: {anomaly["file_idx"]}, Spatial: {anomaly["spatial_idx"]}, '
                  f'Z-Score: {anomaly["z_score"]:.2f}')
    
        # Select 10 evenly spaced points for plotting
        raw_indices = np.linspace(0, 10,len(anomaly['raw_data']) - 1, dtype=int)
        recon_indices = np.linspace(0, 10, len(anomaly['raw_recon']) - 1, dtype=int)
    
        # Plot raw and reconstructed data
        plt.plot(raw_indices, [anomaly['raw_data'][i] for i in raw_indices], label='Raw Data', color='blue')
        plt.plot(recon_indices, [anomaly['raw_recon'][i] for i in recon_indices], label='Reconstructed Data', linestyle='--', color='orange')
        #plt.legend()
    
        # Frequency domain plot
        plt.subplot(2, 1, 2)
        plt.title(f'Anomaly {idx+1} - Frequency Domain')
    
        # Select 10 evenly spaced points for FFT data
        fft_indices = np.linspace(0, len(anomaly['fft_data']) - 1, 10, dtype=int)
        fft_recon_indices = np.linspace(0, len(anomaly['fft_recon']) - 1, 10, dtype=int)
    
        # Plot FFT raw and reconstructed data
        plt.plot(fft_indices, [anomaly['fft_data'][i] for i in fft_indices], label='Raw FFT Data', color='blue')
        plt.plot(fft_recon_indices, [anomaly['fft_recon'][i] for i in fft_recon_indices], label='Reconstructed FFT Data', linestyle='--', color='orange')
        #plt.legend()
    
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'anomaly_{idx+1}.png'))
        plt.close()
    
    # Filter anomalies with Z-score > 3.1
    filtered_anomalies = [anomaly for anomaly in sorted_anomalies if anomaly["z_score"] > 3.1]
    
    # Plot each filtered anomaly
    for idx, anomaly in enumerate(filtered_anomalies):
        plot_anomaly(anomaly, idx)
    
    # Save anomaly details to CSV
    if filtered_anomalies:
        anomaly_df = pd.DataFrame(filtered_anomalies)
        anomaly_df.to_csv(os.path.join(output_dir, 'filtered_anomaly_details.csv'), index=False)
        print(f"Anomaly detection complete. {len(filtered_anomalies)} results saved in {output_dir}")
    else:
        print("No anomalies with Z-score > 3 were found.")



2024-12-06 15:47:47.711274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 15:47:47.728165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 15:47:47.733136: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 15:47:47.746559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Paths to update
hdf5_file_path = 'raw_data.h5'
model_path = 'autoencoder2.keras'

detect_anomalies(hdf5_file_path, model_path)


I0000 00:00:1733525271.182097    6079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733525271.222670    6079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733525271.222772    6079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733525271.229474    6079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733525271.229682    6079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

0


I0000 00:00:1733525277.394393    6140 service.cc:146] XLA service 0x7f598c003d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733525277.394449    6140 service.cc:154]   StreamExecutor device (0): Quadro RTX 3000, Compute Capability 7.5
2024-12-06 15:47:57.416304: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-06 15:47:57.475222: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
I0000 00:00:1733525278.980989    6140 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


Anomaly detection complete. Results saved in anomaly_results


Anomaly detection complete. 4 results saved in anomaly_results
